## Vgg16 model

In [ ]:
import numpy as np
import zipfile
import os
import shutil

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# Extract the provided zip file
with zipfile.ZipFile('archive (3).zip', 'r') as zip_ref:
    zip_ref.extractall('archive (3)')

# Define directories for train, validation, and test sets
train_dir = 'archive (3)/train'
val_dir = 'archive (3)/validation'
test_dir = 'archive (3)/test'

# Create validation directory if it doesn't exist
if not os.path.exists(val_dir):
    os.makedirs(val_dir)

# Define the ratio of data to move to the validation set
validation_split = 0.2

# Move a portion of the training data to the validation directory
for class_name in os.listdir(train_dir):
    class_dir = os.path.join(train_dir, class_name)
    files = os.listdir(class_dir)
    num_validation = int(len(files) * validation_split)
    validation_files = files[:num_validation]
    for file in validation_files:
        src = os.path.join(class_dir, file)
        dst = os.path.join(val_dir, class_name)
        os.makedirs(dst, exist_ok=True)
        shutil.move(src, dst)

# Image dimensions
img_height, img_width = 224, 224
batch_size = 32

# Preprocess and augment data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load pre-trained VGG16 model without top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
model = Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(2, activation='softmax')  # 3 classes for classification
])

# Compile the model
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          epochs=10,
          validation_data=val_generator,
          validation_steps=val_generator.samples // batch_size)

# Evaluate the model
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)


Found 1021 images belonging to 2 classes.
Found 254 images belonging to 2 classes.
58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/10
31/31 [==============================] - 692s 22s/step - loss: 1.8883 - accuracy: 0.6724 - val_loss: 0.2990 - val_accuracy: 0.8393
Epoch 2/10
31/31 [==============================] - 682s 22s/step - loss: 0.3899 - accuracy: 0.8311 - val_loss: 0.3505 - val_accuracy: 0.8482
Epoch 3/10
31/31 [==============================] - 683s 22s/step - loss: 0.3184 - accuracy: 0.8534 - val_loss: 0.3187 - val_accuracy: 0.8438
Epoch 4/10
31/31 [==============================] - 682s 22s/step - loss: 0.2720 - accuracy: 0.8868 - val_loss: 0.2951 - val_accuracy: 0.8616
Epoch 5/10
31/31 [==============================] - 686s 22s/step - loss: 0.2386 - accuracy: 0.8999 - val_loss: 0.3479 - val_accuracy: 0.8304
Epoch 6/10
31/31 [==============================] - 702s 23s/step - loss: 0.2055 - accuracy: 0.9151 - val_loss: 0.3329 - val_accuracy: 0.8571


##LeNet Model

In [ ]:
import numpy as np
import zipfile
import os
import shutil

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Extract the provided zip file
with zipfile.ZipFile('archive (3).zip', 'r') as zip_ref:
    zip_ref.extractall('archive (3)')

# Define directories for train, validation, and test sets
train_dir = 'archive (3)/train'
val_dir = 'archive (3)/validation'
test_dir = 'archive (3)/test'

# Create validation directory if it doesn't exist
if not os.path.exists(val_dir):
    os.makedirs(val_dir)

# Define the ratio of data to move to the validation set
validation_split = 0.2  # You can adjust this ratio as needed

# Move a portion of the training data to the validation directory
for class_name in os.listdir(train_dir):
    class_dir = os.path.join(train_dir, class_name)
    files = os.listdir(class_dir)
    num_validation = int(len(files) * validation_split)
    validation_files = files[:num_validation]
    for file in validation_files:
        src = os.path.join(class_dir, file)
        dst = os.path.join(val_dir, class_name)
        os.makedirs(dst, exist_ok=True)

        # Handle destination file already exists
        dst_file = os.path.join(dst, file)
        if os.path.exists(dst_file):
            # Rename the file before moving it
            base_name, ext = os.path.splitext(file)
            new_name = base_name + '_1' + ext
            dst_file = os.path.join(dst, new_name)

        shutil.move(src, dst_file)

# Image dimensions
img_height, img_width = 32, 32  # LeNet uses smaller input size
batch_size = 32

# Preprocess and augment data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# LeNet Model
model = Sequential([
    Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(16, kernel_size=(5, 5), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(120, activation='relu'),
    Dense(84, activation='relu'),
    Dense(2, activation='softmax')  #  3 classes for classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          epochs=10,
          validation_data=val_generator,
          validation_steps=val_generator.samples // batch_size)

# Evaluate the model
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)


Found 1021 images belonging to 2 classes.
Found 762 images belonging to 2 classes.
Epoch 1/10
31/31 [==============================] - 8s 201ms/step - loss: 0.6740 - accuracy: 0.5915 - val_loss: 0.6697 - val_accuracy: 0.5625
Epoch 2/10
31/31 [==============================] - 6s 185ms/step - loss: 0.6264 - accuracy: 0.6350 - val_loss: 0.6079 - val_accuracy: 0.6630
Epoch 3/10
31/31 [==============================] - 6s 199ms/step - loss: 0.5817 - accuracy: 0.6946 - val_loss: 0.5680 - val_accuracy: 0.7160
Epoch 4/10
31/31 [==============================] - 6s 187ms/step - loss: 0.5455 - accuracy: 0.7270 - val_loss: 0.6071 - val_accuracy: 0.6984
Epoch 5/10
31/31 [==============================] - 6s 186ms/step - loss: 0.5145 - accuracy: 0.7695 - val_loss: 0.6474 - val_accuracy: 0.6386
Epoch 6/10
31/31 [==============================] - 5s 148ms/step - loss: 0.5063 - accuracy: 0.7725 - val_loss: 0.5761 - val_accuracy: 0.7296
Epoch 7/10
31/31 [==============================] - 5s 146ms/step

##AlexNet

In [ ]:
import numpy as np
import zipfile
import os
import shutil
import time

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Extract the provided zip file
with zipfile.ZipFile('archive (3).zip', 'r') as zip_ref:
    zip_ref.extractall('archive (3)')

# Define directories for train, validation, and test sets
train_dir = 'archive (3)/train'
val_dir = 'archive (3)/validation'
test_dir = 'archive (3)/test'

# Create validation directory if it doesn't exist
if not os.path.exists(val_dir):
    os.makedirs(val_dir)

# Define the ratio of data to move to the validation set
validation_split = 0.2  # You can adjust this ratio as needed

# Move a portion of the training data to the validation directory
for class_name in os.listdir(train_dir):
    class_dir = os.path.join(train_dir, class_name)
    files = os.listdir(class_dir)
    num_validation = int(len(files) * validation_split)
    validation_files = files[:num_validation]
    for file in validation_files:
        src = os.path.join(class_dir, file)
        timestamp = str(time.time()).replace('.', '')  # Append a timestamp to the file name
        dst = os.path.join(val_dir, class_name, timestamp + '_' + file)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.move(src, dst)

# Image dimensions
img_height, img_width = 227, 227  # AlexNet uses larger input size
batch_size = 32

# Preprocess and augment data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# AlexNet Model
model = Sequential([
    Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    Conv2D(256, kernel_size=(5, 5), activation='relu'),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    Conv2D(384, kernel_size=(3, 3), activation='relu'),
    Conv2D(384, kernel_size=(3, 3), activation='relu'),
    Conv2D(256, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  #  3 classes for classification
])

# Compile the model
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          epochs=10,
          validation_data=val_generator,
          validation_steps=val_generator.samples // batch_size)

# Evaluate the model
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)



Found 1021 images belonging to 2 classes.
Found 508 images belonging to 2 classes.
Epoch 1/10
31/31 [==============================] - 111s 3s/step - loss: 0.6570 - accuracy: 0.6097 - val_loss: 0.7223 - val_accuracy: 0.5917
Epoch 2/10
31/31 [==============================] - 118s 4s/step - loss: 0.5436 - accuracy: 0.7462 - val_loss: 0.6091 - val_accuracy: 0.7125
Epoch 3/10
31/31 [==============================] - 118s 4s/step - loss: 0.4610 - accuracy: 0.7978 - val_loss: 0.4965 - val_accuracy: 0.7917
Epoch 4/10
31/31 [==============================] - 116s 4s/step - loss: 0.5544 - accuracy: 0.7351 - val_loss: 0.5698 - val_accuracy: 0.7354
Epoch 5/10
31/31 [==============================] - 116s 4s/step - loss: 0.4887 - accuracy: 0.7634 - val_loss: 0.4520 - val_accuracy: 0.7854
Epoch 6/10
31/31 [==============================] - 116s 4s/step - loss: 0.4366 - accuracy: 0.8160 - val_loss: 0.4417 - val_accuracy: 0.7958
Epoch 7/10
31/31 [==============================] - 116s 4s/step - loss

## EfficientNet

In [ ]:
pip install efficientnet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import zipfile
import os
import shutil
import time

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from efficientnet.tfkeras import EfficientNetB0

# Extract the provided zip file
with zipfile.ZipFile('archive (3).zip', 'r') as zip_ref:
    zip_ref.extractall('archive (3)')

# Define directories for train, validation, and test sets
train_dir = 'archive (3)/train'
val_dir = 'archive (3)/validation'
test_dir = 'archive (3)/test'

# Create validation directory if it doesn't exist
if not os.path.exists(val_dir):
    os.makedirs(val_dir)

# Define the ratio of data to move to the validation set
validation_split = 0.2  # You can adjust this ratio as needed

# Move a portion of the training data to the validation directory
for class_name in os.listdir(train_dir):
    class_dir = os.path.join(train_dir, class_name)
    files = os.listdir(class_dir)
    num_validation = int(len(files) * validation_split)
    validation_files = files[:num_validation]
    for file in validation_files:
        src = os.path.join(class_dir, file)
        timestamp = str(time.time()).replace('.', '')  # Append a timestamp to the file name
        dst = os.path.join(val_dir, class_name, timestamp + '_' + file)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.move(src, dst)

# Image dimensions
img_height, img_width = 224, 224  # EfficientNetB0 input size
batch_size = 32

# Preprocess and augment data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'  # Set class_mode to 'binary' for binary classification
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'  # Set class_mode to 'binary' for binary classification
)

# EfficientNetB0 Model (Corrected final layer)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Freeze the pre-trained weights

model = Sequential([
    base_model,
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1)  # No activation function here (Corrected)
])

# Compile the model (Applying sigmoid during compilation)
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          epochs=10,
          validation_data=val_generator,
          validation_steps=val_generator.samples // batch_size)

# Evaluate the model
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'  # Set class_mode to 'binary' for binary classification
)

test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)


Found 1021 images belonging to 2 classes.
Found 2286 images belonging to 2 classes.
Epoch 1/10
31/31 [==============================] - 297s 10s/step - loss: 6.2832 - accuracy: 0.5644 - val_loss: 6.3071 - val_accuracy: 0.5807
Epoch 2/10
31/31 [==============================] - 281s 9s/step - loss: 6.0953 - accuracy: 0.5613 - val_loss: 5.0712 - val_accuracy: 0.5410
Epoch 3/10
31/31 [==============================] - 288s 9s/step - loss: 5.9823 - accuracy: 0.5367 - val_loss: 5.4407 - val_accuracy: 0.5209
Epoch 4/10
31/31 [==============================] - 221s 7s/step - loss: 5.9338 - accuracy: 0.5404 - val_loss: 5.1174 - val_accuracy: 0.5365
Epoch 5/10
31/31 [==============================] - 279s 9s/step - loss: 5.9576 - accuracy: 0.5672 - val_loss: 5.6672 - val_accuracy: 0.5670
Epoch 6/10
31/31 [==============================] - 227s 7s/step - loss: 6.0919 - accuracy: 0.5341 - val_loss: 5.2148 - val_accuracy: 0.5567
Epoch 7/10
31/31 [==============================] - 233s 8s/step - lo

## Inception net

In [ ]:
import numpy as np
import zipfile
import os
import shutil
import time

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3

# Extract the provided zip file
with zipfile.ZipFile('archive (3).zip', 'r') as zip_ref:
    zip_ref.extractall('archive (3)')

# Define directories for train, validation, and test sets
train_dir = 'archive (3)/train'
val_dir = 'archive (3)/validation'
test_dir = 'archive (3)/test'

# Create validation directory if it doesn't exist
if not os.path.exists(val_dir):
    os.makedirs(val_dir)

# Define the ratio of data to move to the validation set
validation_split = 0.2  # You can adjust this ratio as needed

# Move a portion of the training data to the validation directory
for class_name in os.listdir(train_dir):
    class_dir = os.path.join(train_dir, class_name)
    files = os.listdir(class_dir)
    num_validation = int(len(files) * validation_split)
    validation_files = files[:num_validation]
    for file in validation_files:
        src = os.path.join(class_dir, file)
        timestamp = str(time.time()).replace('.', '')  # Append a timestamp to the file name
        dst = os.path.join(val_dir, class_name, timestamp + '_' + file)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.move(src, dst)

# Image dimensions
img_height, img_width = 299, 299  # InceptionV3 input size
batch_size = 32

# Preprocess and augment data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'  # Ensure class_mode is set to 'binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'  # Ensure class_mode is set to 'binary'
)

# InceptionV3 Model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Freeze the pre-trained weights

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Add activation='sigmoid' for binary classification
])

# Compile the model
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          epochs=10,
          validation_data=val_generator,
          validation_steps=val_generator.samples // batch_size)

# Evaluate the model
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'  # Ensure class_mode is set to 'binary'
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)


Found 1021 images belonging to 2 classes.
Found 254 images belonging to 2 classes.
87910968/87910968 [==============================] - 4s 0us/step
Epoch 1/10
31/31 [==============================] - 349s 11s/step - loss: 0.4044 - accuracy: 0.8413 - val_loss: 0.2401 - val_accuracy: 0.9018
Epoch 2/10
31/31 [==============================] - 306s 10s/step - loss: 0.1924 - accuracy: 0.9201 - val_loss: 0.1646 - val_accuracy: 0.9286
Epoch 3/10
31/31 [==============================] - 301s 10s/step - loss: 0.1520 - accuracy: 0.9393 - val_loss: 0.1433 - val_accuracy: 0.9420
Epoch 4/10
31/31 [==============================] - 337s 11s/step - loss: 0.1564 - accuracy: 0.9464 - val_loss: 0.1396 - val_accuracy: 0.9286
Epoch 5/10
31/31 [==============================] - 331s 11s/step - loss: 0.1462 - accuracy: 0.9393 - val_loss: 0.1285 - val_accuracy: 0.9464
Epoch 6/10
31/31 [==============================] - 302s 10s/step - loss: 0.1286 - accuracy: 0.9414 - val_loss: 0.1553 - val_accuracy: 0.9375


## ResNet

In [ ]:
import numpy as np
import zipfile
import os
import shutil
import time

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50

# Extract the provided zip file
with zipfile.ZipFile('archive (3).zip', 'r') as zip_ref:
    zip_ref.extractall('archive (3)')

# Define directories for train, validation, and test sets
train_dir = 'archive (3)/train'
val_dir = 'archive (3)/validation'
test_dir = 'archive (3)/test'

# Create validation directory if it doesn't exist
if not os.path.exists(val_dir):
    os.makedirs(val_dir)

# Define the ratio of data to move to the validation set
validation_split = 0.2  # You can adjust this ratio as needed

# Move a portion of the training data to the validation directory
for class_name in os.listdir(train_dir):
    class_dir = os.path.join(train_dir, class_name)
    files = os.listdir(class_dir)
    num_validation = int(len(files) * validation_split)
    validation_files = files[:num_validation]
    for file in validation_files:
        src = os.path.join(class_dir, file)
        timestamp = str(time.time()).replace('.', '')
        dst = os.path.join(val_dir, class_name, timestamp + '_' + file)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.move(src, dst)

# Image dimensions
img_height, img_width = 224, 224
batch_size = 32

# Preprocess and augment data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

# ResNet50 Model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Freeze the pre-trained weights

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          epochs=10,
          validation_data=val_generator,
          validation_steps=val_generator.samples // batch_size)

# Evaluate the model
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'  # Ensure class_mode is set to 'binary'
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)


Found 1021 images belonging to 2 classes.
Found 508 images belonging to 2 classes.
Epoch 1/10
31/31 [==============================] - 342s 11s/step - loss: 0.7837 - accuracy: 0.5683 - val_loss: 0.5902 - val_accuracy: 0.6833
Epoch 2/10
31/31 [==============================] - 283s 9s/step - loss: 0.6670 - accuracy: 0.6694 - val_loss: 0.5675 - val_accuracy: 0.7146
Epoch 3/10
31/31 [==============================] - 283s 9s/step - loss: 0.5606 - accuracy: 0.7159 - val_loss: 0.5169 - val_accuracy: 0.7542
Epoch 4/10
31/31 [==============================] - 337s 11s/step - loss: 0.5499 - accuracy: 0.7482 - val_loss: 0.5021 - val_accuracy: 0.7729
Epoch 5/10
31/31 [==============================] - 281s 9s/step - loss: 0.5504 - accuracy: 0.7341 - val_loss: 0.4998 - val_accuracy: 0.7729
Epoch 6/10
31/31 [==============================] - 334s 11s/step - loss: 0.5508 - accuracy: 0.7209 - val_loss: 0.4989 - val_accuracy: 0.7729
Epoch 7/10
31/31 [==============================] - 336s 11s/step - 